# V18 (gated 82+83): Sensitivity sweep

Re-runs the preprocessing-variant sweep on the gated (82+83) dataset to see whether robustness improves when outlier sectors are removed.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

sectors_used = [82, 83]
ds = sh.load_dataset()
lc = sh.stitch_pdcsap_sectors(ds, sectors_used)
depth_ppm, depth_err_ppm = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False, preset='extended')
r = session.run('V18')

print(json.dumps({
  'sectors_used': sectors_used,
  'depth_ppm': depth_ppm,
  'depth_err_ppm': depth_err_ppm,
  'status': r.status,
  'flags': list(r.flags),
  'metrics': r.metrics,
}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "depth_err_ppm": 11.153641839855947,
  "depth_ppm": 264.57118442002604,
  "flags": [],
  "metrics": {
    "best_variant_id": "ds1|ol_none|dt_none",
    "depth_spread_iqr_over_median": 0.558524208753999,
    "metric_variance": 0.5721064373377094,
    "n_variants_failed": 0,
    "n_variants_ok": 12,
    "n_variants_total": 12,
    "score_spread_iqr_over_median": 0.5721064373377094,
    "worst_variant_id": "ds5|ol_none|dt_running_median_0.5d"
  },
  "sectors_used": [
    82,
    83
  ],
  "status": "ok"
}
```

</details>


In [ ]:
import json
import matplotlib.pyplot as plt

from bittr_tess_vetter.plotting import plot_sensitivity_sweep

step_id = '27_gated_82_83_v18_sensitivity_sweep'
fname = 'V18_sensitivity_sweep.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(9, 5))
    plot_sensitivity_sweep(r, ax=ax)
    ax.set_title('V18 (gated 82+83): Sensitivity sweep')
    fig.tight_layout()
    fig.savefig(run_path, dpi=160, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=160, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/27_gated_82_83_v18_sensitivity_sweep/V18_sensitivity_sweep.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/27_gated_82_83_v18_sensitivity_sweep/V18_sensitivity_sweep.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/27_gated_82_83_v18_sensitivity_sweep/V18_sensitivity_sweep.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** substantial spread persists (score IQR/median ≈ 0.57; depth IQR/median ≈ 0.56).
- **Why this is useful:** flags signals that are highly sensitive to analysis choices.
- **Interpretation:** gating sectors did not stabilize the detection against small preprocessing changes.
- **Next step:** V19 (gated) alias diagnostics to check harmonic competition and phase-shift events.

</details>
